In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.mlp.mlp_cv_trainer as cv
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [3]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = pd.read_parquet("../artifacts/features/base/tr_df1.parquet")
test_df1 = pd.read_parquet("../artifacts/features/base/test_df1.parquet")

In [13]:
# Check tuning results
study_name = "mlp_v3"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()

ValueError: Cannot evaluate parameter importances with only a single trial.

In [ ]:
# Create OOF and test predictions
importlib.reload(cv)
params = {
    "hidden_dim1": 480,
    "hidden_dim2": 448,
    "batch_size": 64,
    "lr": 0.0005545707560434345,
    "dropout_rate": 0.05,
    "activation": "ReLU"
}
trainer = cv.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(tr_df1, test_df1)
np.save("../artifacts/preds/base/oof_single_1.npy", oof)
np.save("../artifacts/preds/base/test_single_1.npy", test_preds)

In [ ]:
te.send_telegram_message("MLP Training complete!")